In [1]:
import module.feature_engineering as fe
import module.utility_es as es
import module.answer_ext as ans
from gensim.models import Word2Vec, KeyedVectors
import pickle
from nltk.tokenize import word_tokenize
import pandas as pd
import numpy as np
from flask import Flask, request
import json
from nltk.stem.porter import PorterStemmer
from nltk.tag import pos_tag

In [2]:
w2v_model = KeyedVectors.load_word2vec_format("datasets/GoogleEmbedding/GoogleNews-vectors-negative300.bin", binary=True)

In [4]:
questions_clf = pickle.load(open("model/svm_model.sav","rb"))
entities_ext  = pickle.load(open("model/crf_model.sav","rb"))
es_obj = es.connect_elasticsearch()

Connected!


/home/naq/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [5]:
questions_dict = {
    "ALL":   {"source": ["Title","Category","Avatar","Instructions","Ingredients","Serving"
                         ,"Utensils","Nutrition","Description"
                         ,"PrepTime","CookTime","AdditionTime","TotalTime"], "size":1},
    "DIR":   {"source": ["Title","Category","Avatar","Instructions","Serving"], "size":1},
    "NAME":  {"source": ["Title","Avatar"],"size": 5},
    "ING":   {"source": ["Title","Avatar","Ingredients"], "size": 1},
    "TIME":  {"source": ["Title","Avatar","PrepTime","CookTime","AdditionTime","TotalTime"], "size": 1},
    "OBJ":   {"source": ["Title","Avatar","Description"], "size": 1},
    "NUT":   {"source": ["Title","Avatar","Nutrition"], "size": 1},
    "EQUIP": {"source": ["Title","Avatar","Utensils"], "size": 1},
    "QTY":   {"source": ["Title","Avatar","Ingredients","Serving"], "size": 1}
}

In [6]:
app = Flask(__name__) 

@app.route('/')
def index():
    return "Python server" 
 
@app.route('/answering_question', methods = ['POST']) 
def answering_question(): 
    data = request.get_json() 
    question = data["question"] + " ?"    
    return json.dumps(ans.answering_question(w2v_model, question, questions_dict,questions_clf, entities_ext, es_obj))
 
if __name__ == "__main__": 
    app.run(port=8080)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Jun/2021 16:34:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/Jun/2021 16:34:59] "GET /favicon.ico HTTP/1.1" 404 -
/home/naq/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)
127.0.0.1 - - [03/Jun/2021 16:35:34] "POST /answering_question HTTP/1.1" 200 -
/home/naq/anaconda3/lib/python3.8/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.13/security-minimal-setup.html 